# Avito

## Notebook To Do:

#### Translation:
Russian NLP:
- http://www.redhenlab.org/home/the-cognitive-core-research-topics-in-red-hen/the-barnyard/russian-nlp
- https://github.com/named-entity/nltk4russian
- https://github.com/nlpub/russe-evaluation/tree/master/russe/measures/word2vec

#### Data Cleaning:
- Are any nulls imputable?
        
#### Feature Engineering:
- Generate # of words in (text features) column
- Generate date_time features
    - Day of week
- Generate non-text charactor count in text features column
    - "#### BUY NOW ####" ads
    
#### Model Design:
- tfidf feels like the best choice
- svd is a must
- maybe nn but ive been having good luck in my last nlp project with lgbm

#### Elevated Design:
- adding image data
    - Extracting features from images
        - Color
        - Size
        - number of images
        - look into image categorization... is there a way to measure image quality?
        
#### Quality of Life:
- Memory Management
- Commenting Process
- Having Fun

In [47]:
# using the kaggle test data
use_test = True
# train contains 1.5m+ entries, thus just using 25% of data
use_25 = True

## Imports

In [49]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

In [11]:
!ls ./Inputs

periods_test.csv  test.csv          train.csv
periods_train.csv test_active.csv   train_active.csv


In [24]:
train = pd.read_csv('./Inputs/train.csv', parse_dates=["activation_date"])
if use_test: test = pd.read_csv('./Inputs/test.csv', parse_dates=["activation_date"])

In [44]:
if use_25:
    train = train[:round(train.shape[0]/4)]
    test = test[:round(test.shape[0]/4)]

In [17]:
#train_active = pd.read_csv('./Inputs/train_active.csv')
#periods_train = pd.read_csv('./Inputs/periods_train.csv')

## EDA

## Feature Engineering

### Translation

In [55]:
parent_category_name_map = {"Личные вещи" : "Personal belongings",
                            "Для дома и дачи" : "For the home and garden",
                            "Бытовая электроника" : "Consumer electronics",
                            "Недвижимость" : "Real estate",
                            "Хобби и отдых" : "Hobbies & leisure",
                            "Транспорт" : "Transport",
                            "Услуги" : "Services",
                            "Животные" : "Animals",
                            "Для бизнеса" : "For business"}

region_map = {"Свердловская область" : "Sverdlovsk oblast",
            "Самарская область" : "Samara oblast",
            "Ростовская область" : "Rostov oblast",
            "Татарстан" : "Tatarstan",
            "Волгоградская область" : "Volgograd oblast",
            "Нижегородская область" : "Nizhny Novgorod oblast",
            "Пермский край" : "Perm Krai",
            "Оренбургская область" : "Orenburg oblast",
            "Ханты-Мансийский АО" : "Khanty-Mansi Autonomous Okrug",
            "Тюменская область" : "Tyumen oblast",
            "Башкортостан" : "Bashkortostan",
            "Краснодарский край" : "Krasnodar Krai",
            "Новосибирская область" : "Novosibirsk oblast",
            "Омская область" : "Omsk oblast",
            "Белгородская область" : "Belgorod oblast",
            "Челябинская область" : "Chelyabinsk oblast",
            "Воронежская область" : "Voronezh oblast",
            "Кемеровская область" : "Kemerovo oblast",
            "Саратовская область" : "Saratov oblast",
            "Владимирская область" : "Vladimir oblast",
            "Калининградская область" : "Kaliningrad oblast",
            "Красноярский край" : "Krasnoyarsk Krai",
            "Ярославская область" : "Yaroslavl oblast",
            "Удмуртия" : "Udmurtia",
            "Алтайский край" : "Altai Krai",
            "Иркутская область" : "Irkutsk oblast",
            "Ставропольский край" : "Stavropol Krai",
            "Тульская область" : "Tula oblast"}


category_map = {"Одежда, обувь, аксессуары":"Clothing, shoes, accessories",
                "Детская одежда и обувь":"Children's clothing and shoes",
                "Товары для детей и игрушки":"Children's products and toys",
                "Квартиры":"Apartments",
                "Телефоны":"Phones",
                "Мебель и интерьер":"Furniture and interior",
                "Предложение услуг":"Offer services",
                "Автомобили":"Cars",
                "Ремонт и строительство":"Repair and construction",
                "Бытовая техника":"Appliances",
                "Товары для компьютера":"Products for computer",
                "Дома, дачи, коттеджи":"Houses, villas, cottages",
                "Красота и здоровье":"Health and beauty",
                "Аудио и видео":"Audio and video",
                "Спорт и отдых":"Sports and recreation",
                "Коллекционирование":"Collecting",
                "Оборудование для бизнеса":"Equipment for business",
                "Земельные участки":"Land",
                "Часы и украшения":"Watches and jewelry",
                "Книги и журналы":"Books and magazines",
                "Собаки":"Dogs",
                "Игры, приставки и программы":"Games, consoles and software",
                "Другие животные":"Other animals",
                "Велосипеды":"Bikes",
                "Ноутбуки":"Laptops",
                "Кошки":"Cats",
                "Грузовики и спецтехника":"Trucks and buses",
                "Посуда и товары для кухни":"Tableware and goods for kitchen",
                "Растения":"Plants",
                "Планшеты и электронные книги":"Tablets and e-books",
                "Товары для животных":"Pet products",
                "Комнаты":"Room",
                "Фототехника":"Photo",
                "Коммерческая недвижимость":"Commercial property",
                "Гаражи и машиноместа":"Garages and Parking spaces",
                "Музыкальные инструменты":"Musical instruments",
                "Оргтехника и расходники":"Office equipment and consumables",
                "Птицы":"Birds",
                "Продукты питания":"Food",
                "Мотоциклы и мототехника":"Motorcycles and bikes",
                "Настольные компьютеры":"Desktop computers",
                "Аквариум":"Aquarium",
                "Охота и рыбалка":"Hunting and fishing",
                "Билеты и путешествия":"Tickets and travel",
                "Водный транспорт":"Water transport",
                "Готовый бизнес":"Ready business",
                "Недвижимость за рубежом":"Property abroad"}

In [56]:
# not the cleanest translation, but this works

In [57]:
train['region_en'] = train['region'].apply(lambda x : region_map[x])
train['parent_category_name_en'] = train['parent_category_name'].apply(lambda x : parent_category_name_map[x])
train['category_name_en'] = train['category_name'].apply(lambda x : category_map[x])

In [ ]:
if use_test:
    test['region_en'] = test['region'].apply(lambda x : region_map[x])
    test['parent_category_name_en'] = test['parent_category_name'].apply(lambda x : parent_category_name_map[x])
    test['category_name_en'] = test['category_name'].apply(lambda x : category_map[x])

In [59]:
train.isnull().sum()

item_id                         0
user_id                         0
region                          0
city                            0
parent_category_name            0
category_name                   0
param_1                     14998
param_2                    163974
param_3                    215868
title                           0
description                 28920
price                       21423
item_seq_number                 0
activation_date                 0
user_type                       0
image                       28192
image_top_1                 28192
deal_probability                0
region_en                       0
parent_category_name_en         0
category_name_en                0
dtype: int64

### TF-IDF

In [54]:
train['title']

0                                 Кокоби(кокон для сна)
1                                     Стойка для Одежды
2                                        Philips bluray
3                                            Автокресло
4                                        ВАЗ 2110, 2003
5                                           Авто люлька
6         Водонагреватель 100 литров нержавейка плоский
7                                      Бойфренды colins
8                                                Платье
9                     Полу ботиночки замш натур.Бамбини
10                         1-к квартира, 25 м², 2/2 эт.
11                                               Джинсы
12                  Атласы и Контурныя карты за 8 класс
13                                    Монитор acer 18.5
14                     Продаются щенки немецкой овчарки
15                                 Платье женское новое
16                                Chevrolet Lanos, 2008
17                                     Объемная 

In [50]:
tfidf_vec = TfidfVectorizer(ngram_range=(1,1))
train_tfidf = tfidf_vec.transform(train['title'].values.tolist())
if use_test: 
    full_tfidf = tfidf_vec.fit_transform(train['title'].values.tolist()
                                         + test['title'].values.tolist())
    test_tfidf = tfidf_vec.transform(test['title'].values.tolist())

NotFittedError: TfidfVectorizer - Vocabulary wasn't fitted.

### SVD

In [ ]:
n_comp = 2
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')

if test:
    svd_obj.fit(full_tfidf)
else:
    svd_obj.fit(train_tfidf)
    
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
train_svd.columns = ['svd_title_'+str(i+1) for i in range(n_comp)]
train = pd.concat([train, train_svd], axis=1)

if use_test:
    test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
    test_svd.columns = ['svd_title_'+str(i+1) for i in range(n_comp)]
    test = pd.concat([test, test_svd], axis=1)

In [ ]:
del train_tfidf, train_svd, tfidf_vec,
if use_test:  del test_tfidf,  test_svd, full_tfidf

## Model

## Submit to Kaggle